In [ ]:
# %%bash
# pip install transformers==4.27.3
# pip install accelerate bitsandbytes

#### Скачиваем модель

In [ ]:
chunk1_download_url = "https://huggingface.co/Salesforce/blip2-opt-2.7b/resolve/main/pytorch_model-00001-of-00002.bin"
chunk2_download_url = "https://huggingface.co/Salesforce/blip2-opt-2.7b/resolve/main/pytorch_model-00002-of-00002.bin"
index_download_url = "https://huggingface.co/Salesforce/blip2-opt-2.7b/resolve/main/pytorch_model.bin.index.json"
!wget {chunk1_download_url} -O "checkpoints/pytorch_model-00001-of-00002.bin"
!wget {chunk2_download_url} -O "checkpoints/pytorch_model-00002-of-00002.bin"
!wget {index_download_url} -O "checkpoints/pytorch_model.bin.index.json"

--2023-03-26 21:13:54--  https://huggingface.co/Salesforce/blip2-opt-2.7b/resolve/main/pytorch_model-00001-of-00002.bin
Resolving huggingface.co (huggingface.co)... 35.173.225.216, 34.203.133.210, 23.20.207.15, ...
Connecting to huggingface.co (huggingface.co)|35.173.225.216|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/57/7f/577ff0a2689dd13822d06b566f71563587c0983799dca82b88dcb054450c077d/83f4604e9f2c81dace48cbbb245cbe9acadddce7471c17eedc10cd675bf9af62?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27pytorch_model-00001-of-00002.bin%3B+filename%3D%22pytorch_model-00001-of-00002.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1680124435&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzU3LzdmLzU3N2ZmMGEyNjg5ZGQxMzgyMmQwNmI1NjZmNzE1NjM1ODdjMDk4Mzc5OWRjYTgyYjg4ZGNiMDU0NDUwYzA3N2QvODNmNDYwNGU5ZjJjODFkYWNlNDhjYmJiMjQ1Y2JlOWFjYWRkZGNlNzQ3MWMxN2

По чанкам загружаем в память, самые тяжелые линейные слои кладём на диск(лучше в память, но в колабе её слишком мало), их мы будем подгружать динамически во время работы модели

In [ ]:
import torch
weights = {}

for chunk_name in ["checkpoints/pytorch_model-00001-of-00002.bin", "checkpoints/pytorch_model-00002-of-00002.bin"]:
    partial_state_dict = torch.load(chunk_name)
    keys = list(partial_state_dict.keys())
    for key in keys:
        if "language_model" in key and ("fc" in key or "k_proj" in key or "q_proj" in key or "v_proj" in key or "out_proj" in key) and "weight" in key:
            torch.save(partial_state_dict[key], f"linear/{key}")
        else:
            weights[key] = partial_state_dict[key]
        del partial_state_dict[key]

Загружаем модель без самых тяжеловесных слоев (пока что веса инициализированы рандомно)

In [ ]:
from accelerate import init_empty_weights
from transformers import AutoConfig, AutoModelForCausalLM
from transformers import Blip2Processor, Blip2Config, Blip2ForConditionalGeneration
import torch 

processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
checkpoint = "Salesforce/blip2-opt-2.7b"
config = Blip2Config.from_pretrained(checkpoint)

actual_hidden_layers = config.text_config.num_hidden_layers
config.text_config.num_hidden_layers = 0

model = Blip2ForConditionalGeneration(config)

In [ ]:
model = model.cuda()

In [ ]:
model

Blip2ForConditionalGeneration(
  (vision_model): Blip2VisionModel(
    (embeddings): Blip2VisionEmbeddings(
      (patch_embedding): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
    )
    (encoder): Blip2Encoder(
      (layers): ModuleList(
        (0): Blip2EncoderLayer(
          (self_attn): Blip2Attention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=1408, out_features=4224, bias=True)
            (projection): Linear(in_features=1408, out_features=1408, bias=True)
          )
          (layer_norm1): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
          (mlp): Blip2MLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1408, out_features=6144, bias=True)
            (fc2): Linear(in_features=6144, out_features=1408, bias=True)
          )
          (layer_norm2): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
        )
        (1): Blip2EncoderLayer(
          (self_attn): 

Переопределяем линейный слой, который будет загружать веса (с помощью функции load_by_path - можно из памяти, тут реализация с диском, потому что в память в colab не влезло)

In [ ]:
LINEAR_WEIGHTS = {}

def load_by_path(path):
  # return LINEAR_WEIGHTS[path]
    return torch.load(f"linear/{path}").cuda()


class _OffloadedLinearOp(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, saved_weight_path, bias_or_none):
        weight = load_by_path(saved_weight_path)
        ctx._saved_weight_path = saved_weight_path
        ctx._has_bias = bias_or_none is not None
        return torch.nn.functional.linear(input, weight, bias=bias_or_none)

    @staticmethod
    def backward(ctx, grad_output):
        weight = load_by_path(ctx._saved_weight_path)
        grad_input = torch.nn.functional.linear(grad_output, weight.t())
        grad_bias = grad_output.flatten(0, -2).sum(0) if ctx._has_bias else None
        return grad_input, None, grad_bias

In [ ]:
class MyOffloadedLinear(torch.nn.Module):
    def __init__(self, saved_weight_path, bias_or_none):
        super().__init__()
        self.saved_weight_path, self.bias_or_none = saved_weight_path, bias_or_none
    def forward(self, input):
        return _OffloadedLinearOp.apply(input, self.saved_weight_path, self.bias_or_none)

Подставляем наши переопределенные слои в модель

In [ ]:
import transformers

for i in range(actual_hidden_layers):
    prefix = f"language_model.model.decoder.layers.{i}"
    new_layer = transformers.models.opt.modeling_opt.OPTDecoderLayer(config.text_config)
    new_layer.fc1 = MyOffloadedLinear(f"{prefix}.fc1.weight", weights[f"{prefix}.fc1.bias"].cuda())
    new_layer.fc2 = MyOffloadedLinear(f"{prefix}.fc2.weight", weights[f"{prefix}.fc2.bias"].cuda())
    new_layer.self_attn.k_proj = MyOffloadedLinear(f"{prefix}.self_attn.k_proj.weight", weights[f"{prefix}.self_attn.k_proj.bias"].cuda())
    new_layer.self_attn.v_proj = MyOffloadedLinear(f"{prefix}.self_attn.v_proj.weight", weights[f"{prefix}.self_attn.v_proj.bias"].cuda())
    new_layer.self_attn.q_proj = MyOffloadedLinear(f"{prefix}.self_attn.q_proj.weight", weights[f"{prefix}.self_attn.q_proj.bias"].cuda())
    new_layer.self_attn.out_proj = MyOffloadedLinear(f"{prefix}.self_attn.out_proj.weight", weights[f"{prefix}.self_attn.out_proj.bias"].cuda())
    del weights[f"{prefix}.fc1.bias"]
    del weights[f"{prefix}.fc2.bias"]
    del weights[f"{prefix}.self_attn.k_proj.bias"]
    del weights[f"{prefix}.self_attn.q_proj.bias"]
    del weights[f"{prefix}.self_attn.v_proj.bias"]
    del weights[f"{prefix}.self_attn.out_proj.bias"]
    model.language_model.model.decoder.layers.append(new_layer.cuda())
config.text_config.num_hidden_layers = actual_hidden_layers

Загружаем оставшиеся предобученные веса

In [ ]:
model.load_state_dict(weights, strict=False)

<All keys matched successfully>

In [ ]:
del weights

In [ ]:
for param in model.parameters():
  param.requires_grad = False

# model.gradient_checkpointing_enable()

model.eval()

In [ ]:
# import os

# for f in os.listdir("linear"):
#     LINEAR_WEIGHTS[f] = torch.load(f"linear/{f}")

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

Проверка работоспособности

In [ ]:
from PIL import Image
import requests

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(images=image, return_tensors="pt").to(device)

generated_ids = model.generate(**inputs)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
print(generated_text)

two cats laying on a couch next to a remote
